In [2]:
!pip install -U scikit-learn scikit-surprise numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.0.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (25.9 MB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-linux_x86_64.whl size=2138307 sha256=966621d252a95d88c6e082c64d158a54e80346cf77e48ee80671abf92756544c
  Stored in directory: /common/home/as3592/.cache/pip/wheels/20/91/57/2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
import random

In [4]:
p = 0.1
df = pd.read_csv('data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Everywhere I Go,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,The Len Price 3,Man Who Used To Be,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,Mastermind,HARD ROCK 2010


In [5]:
df.columns = df.columns.str.replace('"', '')
df.columns = df.columns.str.replace('name', '')
df.columns = df.columns.str.replace(' ', '')
df.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

In [6]:
df["artist_track"] = df["artist"]  + "-" + df["track"]
df["artist_track"] = df["artist_track"].str.replace(" ", "")
df.head()

,user_id,artist,track,playlist,artist_track
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010,ElvisCostello-(TheAngelsWannaWearMy)RedShoes
1,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010,PaulMcCartney-BandOnTheRun
2,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Everywhere I Go,HARD ROCK 2010,Lissie-EverywhereIGo
3,9cc0cfd4d7d7885102480dd99e7a90d6,The Len Price 3,Man Who Used To Be,HARD ROCK 2010,TheLenPrice3-ManWhoUsedToBe
4,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,Mastermind,HARD ROCK 2010,CocktailSlippers-Mastermind


In [7]:
size = lambda x: len(x)
df_freq = df.groupby(['user_id', 'artist_track']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist_track', 'freq']]
df_freq.head()

,user_id,artist_track,freq
0,00055176fea33f6e027cd3302289378b,5SecondsOfSummer-EnglishLoveAffair,1
1,00055176fea33f6e027cd3302289378b,5SecondsOfSummer-LongWayHome-Acoustic,1
2,00055176fea33f6e027cd3302289378b,"AgainstTheCurrent-Closer,Faster",1
3,00055176fea33f6e027cd3302289378b,DemiLovato-MadeInTheUSA,1
4,00055176fea33f6e027cd3302289378b,FallOutBoy-MySongsKnowWhatYouDidInTheDark(Ligh...,1


In [8]:
artist_track = pd.unique(df['artist_track'].values.ravel())
artist_track = pd.Series(np.arange(len(artist_track)), artist_track)
df_freq["artist_track_id"] = df_freq[['artist_track']].applymap(artist_track.get)

In [9]:
print(len(df_freq["artist_track"].unique()))

627526


In [10]:
print(len(df_freq["artist_track_id"].unique()))

627526


In [11]:
print(len(df_freq["artist_track"]))

1267223


In [12]:
artist_track_df = pd.DataFrame(df_freq["artist_track_id"].unique(), columns = ["artist_track_id"])

In [13]:
artist_track_df["artist_track"] = df_freq["artist_track"].unique()
df_freq.drop('artist_track', axis=1, inplace=True)
df_freq

,user_id,freq,artist_track_id
0,00055176fea33f6e027cd3302289378b,1,133644
1,00055176fea33f6e027cd3302289378b,1,395484
2,00055176fea33f6e027cd3302289378b,1,493055
3,00055176fea33f6e027cd3302289378b,1,181593
4,00055176fea33f6e027cd3302289378b,1,3951
...,...,...,...
1267218,fff77dadf8528083c920b9c018847e8b,1,78132
1267219,fff77dadf8528083c920b9c018847e8b,1,170568
1267220,fff77dadf8528083c920b9c018847e8b,1,58448
1267221,fff77dadf8528083c920b9c018847e8b,1,9767


In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1,5))

In [16]:
df_freq[["freq"]] = scaler.fit_transform(df_freq[["freq"]])
df_freq.head()

,user_id,freq,artist_track_id
0,00055176fea33f6e027cd3302289378b,1.0,133644
1,00055176fea33f6e027cd3302289378b,1.0,395484
2,00055176fea33f6e027cd3302289378b,1.0,493055
3,00055176fea33f6e027cd3302289378b,1.0,181593
4,00055176fea33f6e027cd3302289378b,1.0,3951


In [42]:
df_freq.freq.unique()

array([1.        , 2.33333333, 3.66666667, 5.        ])

In [17]:
reader = Reader(rating_scale=(1, 5))

In [18]:
data = Dataset.load_from_df(df_freq[['user_id', 'artist_track_id', 'freq']], reader)

In [19]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor

In [77]:
from surprise import SVD
from surprise.model_selection import KFold
from surprise import accuracy
# define a cross-validation iterator
kf = KFold(n_splits=5)
algo = SVD()
for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.1041
RMSE: 0.1057
RMSE: 0.1058
RMSE: 0.1051
RMSE: 0.1049


# Let's try to get predictions from SVD

In [20]:
from surprise import SVD
from surprise.model_selection import cross_validate



In [22]:
svd = SVD(verbose=True, n_epochs=50)

In [23]:
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

{'test_rmse': array([0.17067736, 0.16894306, 0.17152288, 0.16716054, 0.16664314]),
 'fit_time': (189.06539011001587,
  188.03691697120667,
  187.76325869560242,
  188.50650811195374,
  190.52603697776794),
 'test_time': (3.061945915222168,
  3.0488712787628174,
  2.605293035507202,
  3.158702850341797,
  3.0367510318756104)}

In [25]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


In [31]:
preds = list()

In [40]:
for id in df_freq.artist_track_id.values:
    preds.append(svd.predict(uid="00055176fea33f6e027cd3306789378b", iid=id))
    

In [33]:
svd.predict(uid='100', iid='133644')

Prediction(uid='100', iid='133644', r_ui=None, est=1.0194304133263572, details={'was_impossible': False})

In [41]:
max([pred.est for pred in preds])

1.974350089349588